In [33]:
print("ok")

ok


In [34]:
%pwd

'c:\\Users\\rautl\\OneDrive\\Desktop\\Generative-ai\\End-to-end-medical-chatbot-generative-ai\\research'

In [15]:
import os
os.chdir("../")

In [16]:
%pwd

'c:\\Users\\rautl\\OneDrive\\Desktop\\Generative-ai\\End-to-end-medical-chatbot-generative-ai'

In [17]:
%pwd

'c:\\Users\\rautl\\OneDrive\\Desktop\\Generative-ai\\End-to-end-medical-chatbot-generative-ai'

In [18]:
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter


In [19]:
#Extract Data From the PDF File And return the pdf with extracted data
def load_pdf_file(data):
    loader= DirectoryLoader(data,
                            glob="*.pdf",
                            loader_cls=PyPDFLoader)

    documents=loader.load()

    return documents

In [20]:
# Load the PDF files from the specified directory and extract their data And then return the document with the extracted data and stored into the extracted data variable
extracted_data = load_pdf_file(data="Data") 



In [21]:
# Split the Extracted_Data into Text Chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=20)
    text_chunks = text_splitter.split_documents(extracted_data)
    return text_chunks

In [22]:
# Split the extracted data into text chunks for processing
text_chunks = text_split(extracted_data)
print("Length of Text Chunks", len(text_chunks))

Length of Text Chunks 6973


In [46]:
# text_chunks

In [23]:
##Create Hugging Face Embeddings 


from langchain_community.embeddings import HuggingFaceEmbeddings





In [24]:
##Download the Embeddings from Hugging Face
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings


In [25]:
embeddings = download_hugging_face_embeddings()

C:\Users\rautl\AppData\Local\Temp\ipykernel_2996\3917360699.py:3: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")


In [26]:




query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [53]:
# query_result

In [27]:
from dotenv import load_dotenv
import os
load_dotenv()

True

In [28]:
PINECONE_API_KEY=os.environ.get('PINECONE_API_KEY')
GROQ_API_KEY=os.environ.get('GROQ_API_KEY')

In [29]:
from pinecone import Pinecone, ServerlessSpec

pc = Pinecone(api_key=PINECONE_API_KEY)

# pc.create_index(
#     name="medicalbot",
#     dimension=384,
#     metric="cosine",
#     spec=ServerlessSpec(
#         cloud="aws",
#         region="us-east-1"
#     )
# )

index_name = "medicalbot"


print(pc.list_indexes())


[{
    "name": "medicalbot",
    "metric": "cosine",
    "host": "medicalbot-kd1um55.svc.aped-4627-b74a.pinecone.io",
    "spec": {
        "serverless": {
            "cloud": "aws",
            "region": "us-east-1"
        }
    },
    "status": {
        "ready": true,
        "state": "Ready"
    },
    "vector_type": "dense",
    "dimension": 384,
    "deletion_protection": "disabled",
    "tags": null
}]


In [30]:
import os

os.environ["PINECONE_API_KEY"] = PINECONE_API_KEY

os.environ["GROQ_API_KEY"] = GROQ_API_KEY



In [31]:
print(extracted_data[:14])


[Document(metadata={'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'moddate': '2017-05-01T10:37:35-07:00', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'total_pages': 759, 'page': 0, 'page_label': '1'}, page_content='The GALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION'), Document(metadata={'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'moddate': '2017-05-01T10:37:35-07:00', 'title': '', 'author': '', 'subject': '', 'keywords': '', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'total_pages': 759, 'page': 1, 'page_label': '2'}, page_content='The G ALE\nENCYCLOPEDIA\nof MEDICINE\nSECOND EDITION\nJACQUELINE L. LONGE, EDITOR\nDEIRDRE S. BLANCHFIELD, ASSOCIATE EDITOR\nVOLUME\nC-F\n2'), Document(metadata={'producer': 'GPL Ghostscript 9.10', 'creator': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'm

In [32]:
from langchain_pinecone import Pinecone

index_name = "medicalbot"

docsearch = Pinecone.from_documents(
    documents=text_chunks,
    embedding=embeddings,
    index_name=index_name
)

In [33]:
# Load Existing index 

from langchain_pinecone import Pinecone

docsearch = Pinecone.from_existing_index(
    index_name=index_name,
    embedding=embeddings
)
# This step reads embeddings already stored inside Pinecone Vector database where all the Embeddings are store
# without needing your PDF or text_chunks.


In [34]:
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_pinecone import Pinecone as PineconeVectorStore

embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")



In [35]:
docs = docsearch.similarity_search("What is this document about?", k=3)
docs


[Document(id='05c1c113-e022-4f30-b6e0-15fed5ffba59', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 2.0, 'page_label': '3', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='er will be corrected in future editions.\nThis book is printed on recycled paper that meets Environmental Pro-\ntection Agency standards.\nThe paper used in this publication meets the minimum requirements of\nAmerican National Standard for Information Sciences-Permanence\nPaper for Printed Library Materials, ANSI Z39.48-1984.\nThis publication is a creative work fully protected by all applicable\ncopyright laws, as well as by misappropriation, trade secret, unfair com-'),
 Document(id='62965c86-9b5d-4126-b257-d6bc41a252c8', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': 

In [36]:
docs = load_pdf_file("Data")
print(len(docs))


759


In [37]:
# This is optional to chequek That everything is work fine it means docsearch fetching the data from the knowledge base pinecone
query = " What is caffeine?"
results = docsearch.similarity_search(query, k=5)
results



[Document(id='2c3d7f38-d1d9-4c40-bb4d-b9be1348fc36', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 13.0, 'page_label': '14', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='CABG surgery see Coronary artery bypass\ngraft surgery\nCAD see Coronary artery disease\nCaffeine\nDefinition\nCaffeine is a drug that stimulates the central nervous\nsystem.\nPurpose\nCaffeine makes people more alert, less drowsy, and\nimproves coordination. Combined with certain pain\nrelievers or medicines for treating migraine headache ,\ncaffeine makes those drugs work more quickly and effec-\ntively. Caffeine alone can also help relieve headaches.\nAntihistamines are sometimes combined with caffeine'),
 Document(id='642681d1-8b19-4d04-bc0b-1c7f71b8558d', metadata={'author': '', 'creationdate': '

In [38]:
caffeine_pages = [d for d in docs if "Caffeine" in d.page_content]
len(caffeine_pages)


5

In [41]:
retriever = docsearch.as_retriever(search_type="similarity", search_kwargs={"k":3})

In [40]:
retrieved_docs = retriever.invoke("What is Acne?")

In [ ]:
# retrieved_docs

[Document(id='c28568b6-b421-4924-97cc-7b3409c7c4d6', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator': '', 'keywords': '', 'moddate': '2017-05-01T10:37:35-07:00', 'page': 425.0, 'page_label': '426', 'producer': 'GPL Ghostscript 9.10', 'source': 'Data\\The_GALE_ENCYCLOPEDIA_of_MEDICINE_SECOND.pdf', 'subject': '', 'title': '', 'total_pages': 759.0}, page_content='Corticosteriod —A group of synthetic hormones\nthat are used to prevent or reduce inflammation.\nToxic effects may result from rapid withdrawal after\nprolonged use or from continued use of large doses.\nPatch test—A skin test that is done to identify aller-\ngens. A suspected substance is applied to the skin.\nAfter 24–48 hours, if the area is red and swollen,\nthe test is positive for that substance. If no reaction\noccurs, another substance is applied. This is con-'),
 Document(id='e4d8c83f-fee2-46d2-baf4-2718dda12f69', metadata={'author': '', 'creationdate': '2017-05-01T10:37:35-07:00', 'creator

In [42]:
from langchain_groq import ChatGroq
llm = ChatGroq(
    model="llama-3.1-8b-instant",
    temperature=0.4,
    max_tokens=500,
    api_key=os.getenv("GROQ_API_KEY")
)


In [43]:
from langchain_groq import ChatGroq
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

system_prompt = (
    "You are an assistant for question-answering tasks. "
    "Use the following pieces of retrieved context to answer "
    "the question. If you don't know the answer, say that you "
    "don't know. Use three sentences maximum and keep the "
    "answer concise."
    "\n\n"
    "{context}"
)


prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)


In [46]:
# 4. Build the final LLM chain (same job as StuffDocumentsChain in old LangChain)
# This takes the prompt → runs it through the LLM → converts output to clean text
parser = StrOutputParser()
chain = prompt | llm | parser

# 5. Full RAG pipeline (same job as RetrievalChain in old LangChain)
# Step 1: Retrieve top 3 most relevant chunks from Pinecone based on user question
# Step 2: Merge those chunks into one single context block
# Step 3: Send {context + question} to the LLM and return its answer
def rag_pipeline(query):
    docs = docsearch.similarity_search(query, k=3)
    context = "\n\n".join([d.page_content for d in docs])
    return chain.invoke({"context": context, "input": query})

# 6. Test the chatbot by asking a question
rag_pipeline(" What are the symptoms of caffeine overdose? ")



'Symptoms of caffeine overdose include excitement, agitation, anxiety, confusion, a sensation of light flashing before the eyes, unusual sensitivity to touch, unusual sensitivity of other senses, ringing in the ears, frequent urination, muscle twitches or tremors, and heart problems.'

In [1]:
# -------------------------------------------------------------
# GROQ LLM SANITY CHECK (WHY THIS BLOCK IS IMPORTANT)
# -------------------------------------------------------------
# Before using Groq inside your RAG pipeline, we MUST test:
#   1. API key loading from .env file
#   2. Authentication with the Groq server
#   3. Whether the selected model exists and is available
#   4. Whether the client can generate a basic response
#
# If this block works → your whole chatbot will work.
# -------------------------------------------------------------

from groq import Groq
import os
from dotenv import load_dotenv

# Load environment variables (GROQ_API_KEY)
load_dotenv()

# Initialize Groq client using the API key from .env
client = Groq(api_key=os.getenv("GROQ_API_KEY"))

# -------------------------------------------------------------
# Send a simple test message to verify:
#   ✔ Correct API key
#   ✔ Correct model name (llama-3.1-8b-instant)
#   ✔ Groq servers responding properly
#   ✔ Client library works
# -------------------------------------------------------------
chat = client.chat.completions.create(
    model="llama-3.1-8b-instant",         # Groq's fast + free model
    messages=[{"role": "user", "content": "Hello"}]
)

# Print the model's response to confirm everything is working
print(chat.choices[0].message.content)

# -------------------------------------------------------------
# If you see a response (like "How can I assist you today?"):
#   ✔ Your Groq LLM is fully functional.
#   ✔ Safe to integrate with RAG pipeline.
# -------------------------------------------------------------







Hello. How can I assist you today?
